![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Retrieval Augmented Generation
</b></font> </br></p>


---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/genai_lib
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

In [ ]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([
                ('markitdown[all]', 'markitdown'),
                'langchain_chroma',
                'langchain_huggingface',
                ('unstructured[all-docs]>=0.11.2', 'unstructured'),
                ])

# 1 | Übersicht
---


<p><font color='black' size="5">
Einführung in Retrieval-Augmented Generation (RAG)
</font></p>



Große Sprachmodelle (LLMs), wie sie in LangChain eingebunden sind, bieten leistungsstarke Möglichkeiten zur Verarbeitung und Analyse umfangreicher Textdaten. Sie eignen sich besonders für Aufgaben, die eine Extraktion und ein tiefgehendes Verständnis von Informationen erfordern, da sie in der Lage sind, auf Grundlage eines gegebenen Dokuments präzise Antworten auf Fragen zu generieren.

Ein wichtiger Ansatz zur **dokumentengestützten** Beantwortung von Fragen in Kombination mit LLMs ist die sogenannte Retrieval-Augmented Generation (RAG). Diese Methode verbindet die Vorteile klassischer Informationsabrufsysteme mit den generativen Fähigkeiten moderner Sprachmodelle.

Durch die Verknüpfung von Informationsabruf und Textgenerierung erhöht RAG die Fähigkeit von LLMs, fundierte und kontextbezogene Antworten zu liefern. Besonders nützlich ist dieser Ansatz, wenn relevante Informationen nicht direkt im vorab trainierten Modellwissen vorhanden sind und daher externe Quellen herangezogen werden müssen. Die Integration von RAG in LangChain ermöglicht eine effektive und flexible Lösung für die Beantwortung von Fragen auf Basis von Dokumenten, was ein differenziertes Verständnis und präzise Antworten in verschiedenen Anwendungsbereichen gewährleistet.


**Praktischer Anwendungsfall: Kundenanfrage**

<img src="https://www.researchgate.net/publication/378467192/figure/fig3/AS:11431281235857152@1712887907223/Zusammenspiel-zwischen-Vektordatenbank-Kundenanfrage-und-LLM.png" width="800" alt="Avatar">



Quelle: [Die_Nutzung_von_ChatGPT_in_Unternehmen](https://www.researchgate.net/publication/378467192_Die_Nutzung_von_ChatGPT_in_Unternehmen_Ein_Fallbeispiel_zur_Neugestaltung_von_Serviceprozessen)

# 2 | RAG-Prozess
---

Der Retrieval-Augmented Generation (RAG) Prozess besteht aus zwei zentralen Teilprozessen –  **Datensammlung** zur Vorbereitung externer Wissensquellen und dem **Abruf & Erweiterung** zur kontextgestützten Beantwortung von Nutzeranfragen – und verbindet damit Information Retrieval und Textgenerierung zu einem leistungsfähigen Gesamtsystem.


<img src="https://raw.githubusercontent.com/ralf-42/Image/main/rag_process.png" width="600" alt="Avatar">

<p><font color='black' size="5">
Data Collection Process
</font></p>

+ Dokuments:
Relevante Texte (z.B. Fachartikel, Handbücher, Webseiten) werden gesammelt.

+ Chunking: Aufteilung der längere Texte in kleinere, zusammenhängende Abschnitte (Chunks).

+ Embedding:
Die Texte werden mithilfe eines Embedding-Modells (z. B. sentence-transformers) in numerische Vektoren umgewandelt.

+ Vector Database:
Die Embeddings und zugehörigen Texte werden in einer Vektordatenbank (z. B. FAISS, ChromaDB) gespeichert, um später effizient durchsucht werden zu können.

<p><font color='black' size="5">
Inference Process
</font></p>




+ User Query:
Der Nutzer stellt eine Anfrage (z. B. eine Frage in natürlicher Sprache).

+ Query Embedding:
Die Frage wird in ein Embedding konvertiert

+ Retriever:
Ein Suchsystem durchsucht eine (externe) Vektordatenbank (z.B. mit Dokumenten) nach relevanten Inhalten zur Anfrage.

+ Documents:
Die gefundenen, relevanten Dokumente (Kontextinformationen) werden gesammelt.


+ Enrich Prompt:
Die Abfrage des Nutzers wird um die gefundenen Dokumente ergänzt.

+ Model Inference:
Das generative Sprachmodell (z.B. GPT) erhält die ursprüngliche Anfrage plus die gefundenen Dokumente als Kontext. Es generiert auf dieser Grundlage eine fundierte Antwort.


<p><font color='black' size="5">
Einschränkungen von RAG
</font></p>



LLM RAG kombiniert Sprachmodelle mit externem Datenabruf, um fehlendes Wissen aus spezialisierten oder proprietären Quellen zu ergänzen. Dies ist besonders nützlich in Bereichen wie Finanzen, Recht oder Technik, wo präzise und aktuelle Informationen erforderlich sind.

Wenn die zusätzlichen Daten jedoch bereits Allgemeinwissen sind, bringt RAG wenig Mehrwert. Da das Basismodell umfassend vortrainiert ist, kann es viele Anfragen ohne externe Erweiterung beantworten. Der unnötige Abruf bekannter Informationen führt zudem zu Ineffizienzen und erhöht den Rechenaufwand.

RAG sollte gezielt eingesetzt werden, insbesondere für domänenspezifische oder proprietäre Daten, die das Basismodell nicht abdeckt. In allgemeinen Wissensbereichen ist es meist überflüssig.

[RAG-Visualizer](https://claude.site/artifacts/e54ef5f2-0ba0-4468-9315-18f7c1c86c4b)

# 3 | Deep Dive: Token & Chunks
---

<p><font color='black' size="5">
Tokenizing
</font></p>

**Warum sind Tokenisierung und Chunking wichtig?**

Große Sprachmodelle (LLMs) wie GPT-3 oder BERT verarbeiten Text nicht als ganze Sätze oder Absätze, sondern als eine Folge von "Tokens". Tokenisierung und Chunking sind entscheidende Vorverarbeitungsschritte, die es ermöglichen, Texte effizient zu verarbeiten und die Leistung von KI-Modellen zu optimieren.



**Tokenisierung: Die Grundlage der Textverarbeitung**

Tokenisierung ist der Prozess, bei dem Text in kleinere Einheiten, sogenannte Tokens, zerlegt wird. Diese Tokens können Wörter, Teilwörter oder sogar einzelne Zeichen sein.

**Warum ist Tokenisierung wichtig?**

1. **Einheitliche Verarbeitung:** LLMs haben eine begrenzte Eingabelänge (z.B. 512 oder 1024 Tokens). Die Tokenisierung stellt sicher, dass Texte in einem einheitlichen Format vorliegen, das vom Modell verarbeitet werden kann.

2. **Bedeutungserfassung:** Viele Tokens repräsentieren semantische Einheiten, was dem Modell hilft, die Bedeutung des Textes besser zu erfassen.

3. **Effiziente Verarbeitung:** Tokenisierte Texte können effizienter verarbeitet und in numerische Vektoren umgewandelt werden, was für die Eingabe in neuronale Netze notwendig ist.


**Hilfe:**

1 DIN A4 Seite hat ca. 300 Worte und ca. 450 Token

[OpenAI Tokenizer](https://platform.openai.com/tokenizer)

<p><font color='black' size="5">
Chunking
</font></p>

**Chunking: Texte in verdauliche Häppchen teilen**

Chunking ist der Prozess, bei dem längere Texte in kleinere, zusammenhängende Abschnitte (Chunks) aufgeteilt werden.

**Warum ist Chunking wichtig?**

1. **Verarbeitung langer Texte:** Da LLMs eine begrenzte Eingabelänge haben, ermöglicht Chunking die Verarbeitung von Texten, die länger als diese Grenze sind.

2. **Kontexterhaltung:** Durch geschicktes Chunking kann der relevante Kontext innerhalb eines Chunks erhalten bleiben, was für viele NLP-Aufgaben entscheidend ist.

3. **Effizienz:** Kleinere Chunks können parallel verarbeitet werden, was die Gesamtverarbeitungszeit reduzieren kann.



**Hauptansätze für Tokenisierung und Chunking**

1. **Wortbasierte Tokenisierung:** Teilt Text an Wortgrenzen. Einfach, aber nicht immer optimal für komplexe Sprachen oder technische Texte.

2. **Subword-Tokenisierung:** Zerlegt Wörter in häufig vorkommende Teilwörter. Beispiele sind BPE (Byte-Pair Encoding) oder WordPiece. Dies ist besonders nützlich für die Behandlung von unbekannten Wörtern und morphologisch reichen Sprachen.

3. **Zeichenbasierte Tokenisierung:** Betrachtet jedes Zeichen als separates Token. Nützlich für bestimmte Sprachen oder spezielle Anwendungen.

4. **Satzbasiertes Chunking:** Teilt Text in Sätze. Einfach und oft effektiv, kann aber Kontext zwischen Sätzen verlieren.

5. **Überlappende Chunks:** Erstellt Chunks mit Überlappungen, um Kontext an den Chunk-Grenzen zu erhalten.

6. **Semantisches Chunking:** Versucht, Chunks basierend auf Bedeutungseinheiten zu erstellen, was komplexer, aber oft effektiver für das Verständnis ist.

Durch die richtige Kombination von Tokenisierung und Chunking können wir Texte so vorbereiten, dass sie optimal von KI-Modellen verarbeitet werden können. Dies verbessert nicht nur die Leistung der Modelle, sondern ermöglicht auch die Verarbeitung von Texten beliebiger Länge in RAG-Systemen und anderen NLP-Anwendungen.

[ChunkViz](https://chunkviz.up.railway.app/)

# 4 | Deep Dive: Embedding
---

Einbettungen stellen eine KI-optimierte Repräsentation verschiedener Datentypen dar und eignen sich daher besonders für den Einsatz in einer Vielzahl KI-gestützter Tools und Algorithmen. Sie erfassen die wesentlichen Merkmale von Texten, Bildern und auch von Audio- und Videodaten.

Ein Einbettungsmodell verarbeitet Eingangsdaten und wandelt sie in numerische Vektoren um. Die Architektur des Modells sorgt dafür, dass ähnliche Inhalte – beispielsweise Texte mit verwandter Bedeutung oder visuell ähnliche Bilder – im Vektorraum näher beieinander liegen, während sich unähnliche Daten weiter voneinander entfernt befinden.

Ein **Embedding** ist ein Vektor aus Gleitkommazahlen, der Ähnlichkeiten zwischen Texten misst. Kleinere Abstände zwischen Vektoren bedeuten eine stärkere inhaltliche Nähe.

OpenAI bietet zwei Einbettungsmodelle an:
- **text-embedding-3-small**
- **text-embedding-3-large**

**Unterschiede und Auswahlkriterien:**

| Kriterium         | text-embedding-3-small | text-embedding-3-large |
|------------------|----------------------|----------------------|
| **Genauigkeit & Leistung** | Weniger detailliert, aber für viele Aufgaben ausreichend | Erfasst komplexere Zusammenhänge, ideal für anspruchsvolle NLP-Aufgaben |
| **Rechenaufwand** | Effizient, benötigt weniger Ressourcen | Höherer Speicher- und Rechenbedarf |
| **Latenz & Geschwindigkeit** | Schnellere Verarbeitung, geringe Latenz | Höhere Latenz, nicht ideal für Echtzeit-Anwendungen |
| **Kosten** | Kostengünstiger, ideal für skalierbare Anwendungen | Teurer in Betrieb und Bereitstellung |
| **Anwendungsfälle** | Chatbots, Echtzeitsysteme, einfache Textverarbeitung | Semantische Analyse, komplexe NLP-Aufgaben, KI-Forschung |



**Fazit**:  
- **Large**: Wenn hohe Präzision, detailliertes Sprachverständnis und ausreichend Ressourcen vorhanden sind.  
- **Small**: Wenn Effizienz, niedrige Kosten und schnelle Reaktionszeiten im Vordergrund stehen.  

Die Wahl des Modells hängt von den spezifischen Anforderungen der Anwendung ab.


<p><font color='black' size="5">
Instanz eines Einbettungsmodells
</font></p>

Hier ist ein Beispiel für die Erstellung einer Instanz mit text-embedding-3-small.

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

<p><font color='black' size="5">
Vektoren
</font></p>

Ein Vektor ist eine mathematische Darstellung, die eine Menge von Zahlen in einer bestimmten Reihenfolge speichert. In einem Einbettungsmodell wird jeder Text – sei es ein einzelnes Wort, ein Satz oder ein ganzer Absatz – als Vektor in einem hochdimensionalen Raum dargestellt. Diese Darstellung ermöglicht es, Ähnlichkeiten zwischen Texten mathematisch zu berechnen.

**Wortvektoren**
Wenn ein Einbettungsmodell ein einzelnes Wort in einen Vektor umwandelt, geschieht dies auf Basis der Bedeutung und des Kontexts des Wortes. Das bedeutet, dass Wörter mit ähnlicher Bedeutung oder Verwendung in der Sprache ähnliche Vektoren erhalten. Ein Beispiel:

- Das Wort „Katze“ könnte in einem 3D-Raum (vereinfacht dargestellt) als Vektor **(0.5, 1.2, -0.3)** erscheinen.
- Das Wort „Hund“ könnte einen ähnlichen Vektor haben, z. B. **(0.6, 1.1, -0.2)**, da beide Begriffe semantisch verwandt sind.

Dagegen hätte ein völlig anderes Wort wie „Auto“ einen weit entfernten Vektor, z. B. **(2.3, -0.4, 1.7)**, da es eine ganz andere Bedeutung hat.


In [ ]:
vektor1 = embeddings_model.embed_query("Hund")
vektor2 = embeddings_model.embed_query("Ein Hund läuft über die Straße.")

print(type(vektor1))

Die Ausgabe besteht lediglich aus einer gewöhnlichen Python-Liste, die jedoch eine erhebliche Länge aufweist.

In [ ]:
print(len(vektor1))
print(len(vektor2))

Die Länge dieser Zeichenfolge bleibt bei allen Abfragen desselben Modells konstant. Obwohl die größere Modellversion in der Lage ist, qualitativ bessere Vektoren zur Unterscheidung von Zeichenfolgen zu erzeugen, bedeutet dies nicht zwangsläufig, dass eine größere Vektorlänge erforderlich ist, um diese Verbesserung zu erzielen. Dieses Konzept ist komplex und bedarf weiterer Untersuchung.

Betrachtet man die eigentliche Liste, erkennt man, dass sie lediglich eine Sammlung von Zahlen enthält. Nachfolgend sind die ersten zehn Elemente dargestellt.

In [ ]:
print(vektor1[:5]) # Hund
print(vektor2[:5]) # Ein Hund läuft über die Straße.

<p><font color='black' size="5">
 Vektoren vergleichen
</font></p>

In der Mathematik gibt es verschiedene Methoden, um Vektoren miteinander zu vergleichen. Einige der gängigsten Ansätze sind:


* **Kosinus-Ähnlichkeit**: Berechnet den Kosinus des Winkels zwischen zwei Vektoren, wobei die Orientierung stärker gewichtet wird als die Größe. Diese Technik ist besonders verbreitet in Bereichen wie Text Mining und der Informationssuche, um Dokumente auf Ähnlichkeit zu prüfen.
* **Euklidische Distanz**: Gibt die direkte Entfernung zwischen zwei Vektoren an und wird häufig in Clustering-Methoden oder Algorithmen zum nächsten Nachbarn im maschinellen Lernen verwendet.

* **Manhattan-Distanz**: Addiert die absoluten Differenzen der Komponenten zweier Vektoren. Diese Methode wird vor allem in gitterbasierten Pfadsuchalgorithmen sowie in bestimmten maschinellen Lernanwendungen eingesetzt.

Laut einer Empfehlung von OpenAI [FAQ](https://platform.openai.com/docs/guides/embeddings/frequently-asked-questions), eignet sich die Kosinus-Ähnlichkeit besonders gut zum Vergleich von Vektoren, da dabei die ursprüngliche Größe beibehalten wird. Zudem bleibt das Vorzeichen der einzelnen Vektorkomponenten erhalten, was für die Analyse entscheidend ist.

**Interpretation:**

| Ähnlichkeitsmaß         | Wertebereich | Interpretation                                                                                           |
|-------------------------|--------------|---------------------------------------------------------------------------------------------------------|
| Kosinus-Ähnlichkeit     | -1 bis 1     | - 1: Maximale Ähnlichkeit (identische Richtung) <br> - 0: Keine Ähnlichkeit (orthogonal) <br> - -1: Maximale Unähnlichkeit (entgegengesetzte Richtung) <br> - Höhere Werte bedeuten größere Ähnlichkeit |
| Euklidischer Abstand     | 0 bis ∞      | - 0: Identische Vektoren (maximale Ähnlichkeit) <br> - Je größer der Wert, desto unähnlicher sind die Vektoren <br> - Niedrigere Werte bedeuten größere Ähnlichkeit |
| Manhattan-Distanz       | 0 bis ∞      | - 0: Identische Vektoren (maximale Ähnlichkeit) <br> - Je größer der Wert, desto unähnlicher sind die Vektoren <br> - Niedrigere Werte bedeuten größere Ähnlichkeit |


In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Ähnlichkeit ermitteln</font> </br></p>
# Ermittlung & Ausgabe von Vektoren
def similarity(vektor1, vektor2):
    import numpy as np
    from scipy.spatial.distance import cosine, euclidean, cityblock
    # Zwei Beispiel-Einbettungsvektoren
    vector1 = np.array(vektor1)
    vector2 = np.array(vektor2)

    # 1. Kosinus-Ähnlichkeit
    cosine_similarity = 1 - cosine(vector1, vector2)
    print(f"Kosinus-Ähnlichkeit: {cosine_similarity:.4f}")

    # # 2. Euklidischer Abstand
    # euclidean_distance = euclidean(vector1, vector2)
    # print(f"Euklidischer Abstand: {euclidean_distance:.4f}")

    # # 3. Manhattan-Distanz
    # manhattan_distance = cityblock(vector1, vector2)
    # print(f"Manhattan-Distanz: {manhattan_distance:.4f}")

In [ ]:
vektor1 = embeddings_model.embed_query("Ein Hund läuft über eine Brücke")
vektor2 = embeddings_model.embed_query("Ein Hund läuft über die Straße.")
similarity(vektor1, vektor2)

In [ ]:
vektor1 = embeddings_model.embed_query("Ein Hund läuft über eine Brücke.")
vektor2 = embeddings_model.embed_query("Quantenmechanik beschreibt das Verhalten subatomarer Teilchen.")
similarity(vektor1, vektor2)

[Embedding Projector](https://projector.tensorflow.org/?hl=de)

# 5 | Deep Dive: Vectorstore
---

Ein **Vectorstore** ist eine spezialisierte Datenbank zur Speicherung und schnellen Suche von Texten in Form von Vektoren. Er bildet die Grundlage für semantische Suche in Retrieval-Systemen wie RAG. Durch die Umwandlung von Text in numerische Repräsentationen (Embeddings) können inhaltlich ähnliche Informationen effizient gefunden werden – selbst wenn die exakten Wörter nicht übereinstimmen.


**Beispiel Erstellung & Abfrage eines Vectorstore:**

In [ ]:
# Import
import os
from IPython.display import display, Markdown
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.schema import Document

In [ ]:
# Chroma-Speicherverzeichnis (persistent)
chroma_dir = "chroma_demo"

In [ ]:
# OpenAI Embeddings vorbereiten
embedding_model = OpenAIEmbeddings()

In [ ]:
# 2. Beispiel-Dokumente definieren
texte = [
    "Python ist eine beliebte Programmiersprache für Machine Learning.",
    "Gradio ist ein Python-Framework zur Erstellung von KI-Demos.",
    "ChromaDB ist eine Vektordatenbank zur Ähnlichkeitssuche.",
    "LangChain verbindet Sprachmodelle mit externem Wissen und Tools.",
    "Mit Hugging Face können moderne NLP-Modelle einfach genutzt werden.",
    "OpenAI bietet leistungsstarke APIs für Text-, Bild- und Sprachmodelle.",
    "Vektordatenbanken speichern Embeddings für semantische Suchanfragen.",
    "Retrieval-Augmented Generation kombiniert Suche mit Textgenerierung.",
    "Embeddings wandeln Texte in numerische Repräsentationen um.",
    "ChatGPT kann als intelligenter Assistent in Anwendungen integriert werden."
]

docs = [Document(page_content=t) for t in texte]

In [ ]:
# 3. Vektordatenbank mit Chroma erstellen
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=chroma_dir
)

In [ ]:
# Eine Beispiel-Query
query = "Was sind Embeddings?"
docs_retrieved = vectordb.similarity_search(query, k=3)   # Die Top 3 Fundstellen werden bereitgestellt

In [ ]:
# Ausgabe der gefundenen Texte
mprint(f"## 🔍 Gefundene Dokumente")
mprint("---")
mprint(f"**Query:** {query}")
print()
mprint(f"**Quellen:**")
for i, doc in enumerate(docs_retrieved, 1):
    print(f"{i}.  Id: {doc.id:40} Inhalt: {doc.page_content}")

# 6 | Hands-On: RAG - Biografien
---

Retrieval-Augmented Generation (RAG) ist eine innovative Methode zur Verbesserung großer Sprachmodelle (LLMs), indem externe Daten in den Generierungsprozess eingebunden werden. Damit RAG optimal funktioniert, muss es Informationen nutzen, die nicht bereits im Basismodell enthalten sind. Dies ist besonders wichtig, da der größte Vorteil von RAG darin liegt, gezielt aktuelle oder spezialisierte Daten abzurufen, die während des ursprünglichen Trainingsprozesses nicht berücksichtigt wurden.

Diese Eigenschaft macht RAG besonders nützlich für Unternehmen, da sie große Mengen interner und proprietärer Daten verwalten, darunter Geschäftsdokumente, Kundendaten und detaillierte Berichte. Da diese Informationen in der Regel nicht öffentlich zugänglich sind, wären sie ohne RAG nicht für ein LLM nutzbar. Durch den Einsatz dieser Technik können Unternehmen ihre eigenen Daten einbinden, um genauere und relevantere Antworten zu generieren, was Entscheidungsprozesse optimiert und die betriebliche Effizienz steigert.

Zur Veranschaulichung der RAG-Fähigkeiten dient ein eigens erstellter Beispieldatensatz mit synthetischen Biografien von Forscher:innen aus unterschiedlichen Forschungsgebieten. Dieser Datensatz demonstriert, wie RAG gezielt Informationen abruft, die ein Basismodell von sich aus nicht enthalten würde, und diese in den Antwortprozess integriert. Damit wird ein praxisnaher Anwendungsfall für den Einsatz von RAG in Unternehmen dargestellt.

In [ ]:
# Importe
# Standardbibliotheken
import os
import re
import warnings
from IPython.display import display, Markdown

# Drittanbieterbibliotheken
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import (
    UnstructuredMarkdownLoader,
    UnstructuredWordDocumentLoader,
    PyPDFLoader,
    UnstructuredFileLoader
)
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma

from langchain import hub

# Warnungen ausschalten
warnings.filterwarnings('ignore')


Die Funktion verarbeitet ein großes Dokument, indem sie es in kleinere, handhabbare Segmente, sogenannte Chunks, unterteilt, um Einbettungen für den LLM-Abruf in Retrieval-Augmented Generation (RAG) zu erstellen. Der Chunk-Parameter bestimmt die maximale Länge jedes Segments und stellt sicher, dass der Text in Teile zerlegt wird, die vom Sprachmodell effizient verarbeitet und analysiert werden können. Der Overlap-Parameter gibt die Anzahl der Token an, die am Anfang jedes neuen Chunks wiederholt werden sollen, wodurch eine Überlappung zwischen aufeinanderfolgenden Chunks entsteht. Diese Überlappung hilft dabei, die kontextuelle Kontinuität über Chunks hinweg aufrechtzuerhalten und verbessert so die Qualität und Genauigkeit der Einbettungen. Durch die systematische Segmentierung des Dokuments und die Generierung von Einbettungen für jeden Chunk verbessert die Funktion die Fähigkeit des Sprachmodells, relevante Informationen abzurufen, was zu präziseren und kontextuell angemessenen Antworten im RAG-Framework führt.

Textdateien mit Biografielisten werden verarbeitet, indem sie in handhabbare Blöcke unterteilt werden. Eine Blockgröße von 1000 Token erleichtert dem Sprachmodell die Erstellung von Einbettungen, während eine Überlappung von 200 Token den Kontext zwischen den Blöcken erhält.

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/rag_process_01.png" width="600" alt="Avatar">


In [ ]:
!rm -rf files
!mkdir files
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_1.txt -o files/biografien_1.txt
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_2.md -o files/biografien_2.md
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_3.pdf -o files/biografien_3.pdf
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_4.docx -o files/biografien_4.docx

In [ ]:
# Loader-Konfiguration
loader_mapping = {
    "*.md": UnstructuredMarkdownLoader,
    "*.docx": UnstructuredWordDocumentLoader,
    "*.pdf": PyPDFLoader,
    "*.txt": UnstructuredFileLoader,  # Loader für .txt Dateien
}

# Funktion zum Laden der Dokumente
def load_documents_from_directory(directory_path):
    """Lädt Dokumente aus dem angegebenen Verzeichnis basierend auf den unterstützten Dateitypen."""
    documents = []
    for file_pattern, loader_cls in loader_mapping.items():
        loader = DirectoryLoader(directory_path, glob=file_pattern, loader_cls=loader_cls)
        documents.extend(loader.load())
    return documents

# Dokumente laden
directory_path = "/content/files"

documents = load_documents_from_directory(directory_path)

In [ ]:
type(documents), len(documents)

Die geladenen Biografiedaten werden in die ChromaDB-Datenbank integriert. ChromaDB setzt standardmäßig das Einbettungsmodell `all-MiniLM-L6-v2` ein, wobei auch andere [Model Wrapper](https://docs.trychroma.com/integrations/openai) zur Verfügung stehen. Um die Dokumente in verarbeitbare Segmente zu unterteilen, kommt der RecursiveCharacterTextSplitter aus der Bibliothek langchain_text_splitters zum Einsatz. Für die Einbettungsfunktionen wird das Modell über SentenceTransformerEmbeddings instanziiert. Chroma dient als Vektorspeicher und arbeitet nahtlos mit diesen Komponenten zusammen, um Einbettungen effizient zu speichern und abzufragen.

In [ ]:
# Text-Splitter konfigurieren und Dokumente aufteilen
chunk_size = 900
chunk_overlap = 300

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_overlap, chunk_overlap=chunk_overlap)
docs = text_splitter.split_documents(documents)

In [ ]:
type(docs), len(docs)

In [ ]:
# Embeddingsmodell festlegen
embedding_model = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embedding_model)

In [ ]:
# Vektordatenbank erstellen und speichern
persistent_directory = "/content/chroma_db"
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=persistent_directory)

Hier wird ein vordefinierter Prompt für RAG-Anwendungen aus dem LangChain Hub geladen. "rlm/rag-prompt" ist ein spezieller Prompt, der für Retrieval-Augmented Generation optimiert ist.


[LangChainHub](https://smith.langchain.com/hub/rlm/rag-prompt)

In [ ]:
rag_prompt = hub.pull("rlm/rag-prompt")

Diese Funktion nimmt eine Liste von Dokumentobjekten (wie sie vom Retriever zurückgegeben werden) und:

+ Extrahiert den Textinhalt (page_content) aus jedem Dokument
+ Verbindet alle Textinhalte mit doppelten Zeilenumbrüchen zu einem einzigen String
+ Dies ist wichtig, um die abgerufenen Dokumente in ein Format zu bringen, das dem LLM als Kontext übergeben werden kann

In [ ]:
def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

Hier werden zwei Hauptkomponenten initialisiert:

+ Ein Large Language Model (LLM) mit OpenAI
+ Ein Retriever, der aus einem vectorstore erstellt wird (der vorher im Code definiert wurde)

Dieser Retriever sucht relevante Dokumente aus dem Vektorspeicher basierend auf Ähnlichkeit zur Abfrage.

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/rag_process_02.png" width="600" alt="Avatar">


In [ ]:
# Festlegen LLM und Retriever
model_name = 'gpt-4o-mini'
temperature = 0
llm = ChatOpenAI(model=model_name, temperature=temperature)

retriever = vectorstore.as_retriever()

**Parameter:**


| Parameter         | Wirkung                         |
| ----------------- | ------------------------------- |
| `k`               | Anzahl der Dokumente im Kontext, default: K=4|
| `score_threshold` | Qualitätsfilter (optional)      |




Beispiel:

```Python
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 10, "score_threshold": 0.7}
)
```

Hier wird die komplette RAG-Pipeline zusammengebaut:

Eingabe-Dictionary wird erstellt mit:

+ "context": Führt zuerst den Retriever aus, um relevante Dokumente zu finden, und formatiert diese dann mit der format_documents-Funktion
+ "question": Leitet die ursprüngliche Frage unverändert weiter (mittels RunnablePassthrough())


Dieses Dictionary wird an den RAG-Prompt übergeben (rag_prompt), der ein Template mit Platzhaltern für Kontext und Frage ist
Der formatierte Prompt wird an das LLM übergeben (llm), das eine Antwort basierend auf der Frage und dem bereitgestellten Kontext generiert
Die LLM-Ausgabe wird durch den StrOutputParser() geleitet, um sie in einen einfachen String zu konvertieren

Die gesamte Pipeline ermöglicht es, eine Frage zu stellen, relevante Dokumente zu finden, diese mit der Frage zu kombinieren und eine fundierte Antwort zu generieren, die auf den abgerufenen Informationen basiert.

In [ ]:
# Chat-Verlauf initialisieren
chat_history = []

chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Die RAG-Kette kann nun aufgerufen werden, um Informationen zu einer der in den Beispielbiografiedaten enthaltenen Personen abzurufen.

Die gesamte Chain wird mit einem Input aufgerufen. Dieser Input standardmäßig an alle Runnable-Komponenten weitergegeben, **wenn sie ihn erwarten**.

In [ ]:
input = "Was macht Tariq Hassan?"
response = chain.invoke(input)

mprint("### 🧑 Mensch:")
mprint(input)
mprint("### 🤖 KI:")
mprint(response)

In [ ]:
input = "Welche Daten sind zu Elara Fontaine verfügbar?"
response = chain.invoke(input)

mprint("### 🧑 Mensch:")
mprint(input)
mprint("### 🤖 KI:")
mprint(response)

In [ ]:
input = "Wer ist Ralf Bendig?"
response = chain.invoke(input)

mprint("### 🧑 Mensch:")
mprint(input)
mprint("### 🤖 KI:")
mprint(response)


<p><font color='black' size="5">
Exkurs: MarkItDown-Loader
</font></p>


MarkItDown ist eine Python-Bibliothek, die die Verarbeitung und Konvertierung verschiedener Dokumenttypen vereinfacht. Sie dient als *universeller* Dokumenten-Loader für RAG-Anwendungen (Retrieval-Augmented Generation) und andere NLP-Systeme.
Hauptmerkmale

+ Universeller Dokumenten-Reader: Lädt verschiedene Dateiformate mit einer einheitlichen Schnittstelle
+ Automatische Formaterkennung: Erkennt Dateitypen ohne manuelle Zuordnung
+ Einfache API: Minimaler Code zum Einlesen ganzer Dokumentverzeichnisse
+ Formatübergreifend: Unterstützt gängige Formate wie PDF, DOCX, MD, TXT und mehr

In [ ]:
from pathlib import Path
from markitdown import MarkItDown

def load_documents_from_directory(directory_path: str) -> list:
    """  Lädt alle Dokumente aus dem angegebenen Verzeichnis und konvertiert in Markdown """
    # MarkItDown Instanz erstellen
    md_converter = MarkItDown()

    # Liste für konvertierte Inhalte
    documents = []

    # Unterstützte Dateierweiterungen
    supported_extensions = {
        '.pdf', '.docx', '.doc', '.xlsx', '.xls', '.pptx', '.ppt',
        '.txt', '.md', '.html', '.htm', '.csv', '.json', '.xml',
        '.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff'
    }

    # Verzeichnis durchlaufen
    directory = Path(directory_path)

    for file_path in directory.rglob('*'):
        if file_path.is_file() and file_path.suffix.lower() in supported_extensions:
            try:
                # Datei konvertieren
                result = md_converter.convert(str(file_path))
                documents.append(result.text_content)
                print(f"✓ Konvertiert: {file_path.name}")

            except Exception as e:
                print(f"✗ Fehler bei {file_path.name}: {e}")
                continue

    return documents

In [ ]:
docs = load_documents_from_directory("/content/files")

In [ ]:
type(docs), len(docs)

**Vergleich: MarkItdown vs. LangChain Loader**



| Kriterium | MarkItdown | LangChain Loader |
|---|---|---|
| Codemenge | ✅ Deutlich weniger Code nötig (ca. 3-5 Zeilen) | ❌ Erfordert mehr Code inkl. Mappings für verschiedene Dateitypen (ca. 10-15 Zeilen) |
| Dateityperkennung | ✅ Automatische Erkennung von Dateiformaten | ❌ Manuelle Zuordnung von Dateitypen zu entsprechenden Loadern erforderlich |
| Anpassungsmöglichkeiten | ❌ Begrenzte formatspezifische Konfigurationsmöglichkeiten | ✅ Umfangreiche Konfigurationsoptionen für jeden Loader |
| LangChain-Integration | ❌ Möglicherweise zusätzliche Konvertierungsschritte nötig | ✅ Nahtlose Integration in die LangChain-Pipeline |
| Fehlerbehandlung | ❌ Möglicherweise weniger detaillierte Fehlermeldungen | ✅ Spezialisierte Fehlerbehandlung je nach Dateityp |
| Performance | ⚠️ Kann bei komplexen Dokumenten langsamer sein | ✅ Optimiert für spezifische Formate |
| Metadaten-Extraktion | ❌ Eingeschränkte Möglichkeiten, formatspezifische Metadaten zu extrahieren | ✅ Umfangreiche Metadaten-Extraktion je nach Dokumenttyp möglich |
| Einstiegshürde | ✅ Einfach zu verstehen und anzuwenden | ❌ Erfordert Kenntnis der verschiedenen Loader-Klassen |
| Community-Support | ⚠️ Möglicherweise weniger verbreitet | ✅ Breite Community und umfangreiche Dokumentation |
| Format-Unterstützung | ⚠️ Abhängig von der Reife des Tools | ✅ Unterstützung für eine Vielzahl spezieller Formate |
| Wartbarkeit | ✅ Weniger Code bedeutet weniger Fehleranfälligkeit | ❌ Mehr Code für die gleiche Funktionalität |
| Zukunftssicherheit | ✅ Neue Dateitypen werden automatisch unterstützt | ❌ Muss manuell um neue Dateitypen erweitert werden |

Markdown eignet sich hervorragend als einheitliches Format für **textdominierte** RAG-Systeme, da es strukturierte Informationen in einer für LLMs leicht verarbeitbaren Form bereitstellt. Die **Stärken** liegen in der Vereinfachung und Standardisierung verschiedener Textquellen.
Bei dokumentenintensiven Anwendungen mit komplexen Formatierungen, umfangreichen Tabellen oder vielen **nicht-textuellen** Inhalten sollten jedoch zusätzliche Mechanismen zur Erhaltung wichtiger Strukturinformationen implementiert werden, um relevante Kontextinformationen nicht zu verlieren.

# 7 | Häufige Probleme und Lösungen
---

| Problem | Symptom | Lösung |
|---------|---------|--------|
| **Niedrige Retrieval-Präzision** | Irrelevante Dokumente gefunden | • Embedding-Modell wechseln<br>• Chunk-Größe anpassen<br>• Hybrid Search verwenden |
| **Halluzinationen** | Fakten nicht in Quellen enthalten | • Temperature reduzieren<br>• Strengere Prompts<br>• Quellentreue in System-Message betonen |
| **Langsame Performance** | Hohe Latenz bei Anfragen | • Vektordatenbank optimieren<br>• Retrieval-Anzahl reduzieren<br>• Caching implementieren |
| **Inkonsistente Antworten** | Verschiedene Antworten auf gleiche Frage | • Temperature auf 0 setzen<br>• Deterministische Retrieval-Reihenfolge<br>• Seed-Parameter verwenden |
| **Context Overflow** | Token-Limit überschritten | • Context Compression<br>• Chunk-Größe reduzieren<br>• Weniger Dokumente abrufen |


# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
RAG zum LLM-Buch
</font></p>

Erstelle Sie ein RAG, um das LLM die ihr eigenes LLM-Buch aus Modul 05 lesen und analysieren zu lassen.

Lassen Sie mehrere Fragen beantworten. Die Antworten sollten so einfach wie möglich sein, z. B. „ja/nein“, der Name einer Stadt oder eine kurze Rollenbeteichnung. Stelle die Ergebnisse in einer Tabelle im Markdown-Format dar.



<p><font color='black' size="5">
Einfache RAG-Evaluation
</font></p>

Bewerten Sie, wie gut ein RAG-System eine bestimmte Frage beantworten kann.
Aufgabe:

+ Erstellen Sie einen kleinen Datensatz mit Informationen zu einem bekannten Thema (z.B. ein Filmzusammenfassung)
+ Formulieren Sie drei konkrete Fragen und die erwarteten Antworten
+ Implementieren Sie einen RAG-Workflow und testen Sie, wie gut die Antworten mit der Erwartung übereinstimmen





**Filmzusammenfassung als Beispiel**   
> Der Herr der Ringe   
Jurassic Park   
Avengers: Endgame

In [ ]:
#
# Kopiervorlage
#
filmbeschreibungen = [
    """Der Herr der Ringe: Die Gefährten" ist ein Fantasy-Film aus dem Jahr 2001, basierend auf dem ersten Band von J.R.R. Tolkiens Trilogie.Der Film handelt von dem Hobbit Frodo Beutlin, der einen mächtigen Ring erbt. Der Zauberer Gandalf entdeckt, dass es sich um den Einen Ring des dunklen Herrschers Sauron handelt. Um Mittelerde zu retten, muss der Ring im Feuer des Schicksalsberges in Mordor zerstört werden. Frodo macht sich zusammen mit acht Gefährten auf die gefährliche Reise: dem Zauberer Gandalf, den Menschen Aragorn und Boromir, dem Elben Legolas, dem Zwerg Gimli und den Hobbits Sam, Merry und Pippin. Regie führte Peter Jackson. Der Film gewann vier Oscars.""",

    """Jurassic Park" ist ein Science-Fiction-Abenteuerfilm aus dem Jahr 1993, basierend auf dem Roman von Michael Crichton. Der Film erzählt die Geschichte eines Freizeitparks, in dem durch Gentechnik lebende Dinosaurier erschaffen wurden. Der Milliardär John Hammond lädt ein Team von Experten auf die Insel Isla Nublar ein, um den Park vor der Eröffnung zu begutachten. Als das Sicherheitssystem versagt, geraten die Besucher in Lebensgefahr, denn die Dinosaurier brechen aus. Die Gruppe muss ums Überleben kämpfen und einen Weg finden, von der Insel zu entkommen. Regie führte Steven Spielberg. Der Film gewann drei Oscars.""",

    """Avengers: Endgame" ist ein Superheldenfilm aus dem Jahr 2019, basierend auf den Marvel-Comics. Der Film schließt an die Ereignisse von „Avengers: Infinity War“ an, in dem der Schurke Thanos die Hälfte allen Lebens im Universum ausgelöscht hat. Die überlebenden Avengers schmieden einen Plan, um mithilfe der Zeitreise die sogenannten Infinity-Steine zu sammeln und den Schaden rückgängig zu machen. Dabei stellen sie sich erneut Thanos in einer epischen Schlacht. Viele Helden kehren zurück, um gemeinsam das Universum zu retten. Regie führten Anthony und Joe Russo. Der Film war ein weltweiter Kassenerfolg und erhielt eine Oscar-Nominierung für die besten visuellen Effekte."""
]